# PyCity Schools Analysis 


* Those attending charter schools had a signifantly higher passing rate that district schools (~95% vs. ~73%).


* Those attending a small to medium sized school had a significantly higher passing rate than large schools (~95% vs .~75%).  This finding parallels the prior charter vs. district finding as most charter schools are smaller in student size.


* Passing reading scores are generally higher than math across all schools, however, the disparity between passing reading scores and passing math scores at district schools is significant ~14% (~80% passing reading vs. ~66% passing math).  The difference is even more evident when comparing the same evaluation vs charter schools ~3% (~96% passing reading vs. ~93% passing math).


* Within each school, there is little difference in averages scores across all grades.  

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Variables to data files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into data frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Enhance the Student data by adding boolean columns to represent passing marks for reading and math
student_data["reading_pass"] =  np.where(student_data['reading_score'] >= 70, 1, 0);
student_data["math_pass"] =  np.where(student_data['math_score'] >= 70, 1, 0);

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Export combined data for practice and outside confirmation of results
#school_data_complete.to_csv('Resources/combined.csv')

## District Summary

In [2]:
# create variables to store summary info
cnt_schools = len(school_data)
cnt_students = len(school_data_complete)
tot_budget = school_data['budget'].sum()
avg_math = student_data['math_score'].mean()
avg_reading = student_data['reading_score'].mean()
pass_rate = ((avg_math + avg_reading) / 2)
cnt_math_pass = student_data['math_pass'].sum()
math_pass_pct = cnt_math_pass / cnt_students * 100
cnt_reading_pass = student_data['reading_pass'].sum()
reading_pass_pct = cnt_reading_pass / cnt_students * 100

# create new summary data set
sum_data = pd.DataFrame({"Total Schools": [cnt_schools],
                         "Total Students": [cnt_students],
                         "Total Budget": [tot_budget],
                         "Average Math Score": [avg_math],
                         "Average Reading Score": [avg_reading],
                         "Passing Math": [math_pass_pct],
                         "Passing Reading": [reading_pass_pct],
                         "Overall Passing Rate":[pass_rate]})

# apply formatting
sum_data["Total Schools"] = sum_data["Total Schools"].astype(float).map("{:,.0f}".format)
sum_data["Total Students"] = sum_data["Total Students"].astype(float).map("{:,.0f}".format)
sum_data["Total Budget"] = sum_data["Total Budget"].astype(float).map("${:,.0f}".format)
sum_data["Average Math Score"] = sum_data["Average Math Score"].astype(float).map("{:.3f}".format)
sum_data["Average Reading Score"] = sum_data["Average Reading Score"].astype(float).map("{:.3f}".format)
sum_data["Passing Math"] = sum_data["Passing Math"].astype(float).map("{:.3f}%".format)
sum_data["Passing Reading"] = sum_data["Passing Reading"].astype(float).map("{:.3f}%".format)
sum_data["Overall Passing Rate"] = sum_data["Overall Passing Rate"].astype(float).map("{:.3f}%".format)

# display results
sum_data.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.985,81.878,74.981%,85.805%,80.432%


## School Summary

In [3]:
# using the merged data, create a grouping based on school attributes
school_grp = school_data_complete.groupby(["school_name", "type", "size", "budget"])

# create a new data series based on the grouped data with sums of math and readings scores
school_df = pd.DataFrame(school_grp[["math_score","reading_score","math_pass", "reading_pass"]].sum())
school_df.reset_index(inplace=True)

# rename columns
school_df.rename(index=str, inplace=True, columns={
    "school_name": "School Name",
    "type": "School Type",
    "size": "Total Students",
    "budget": "Total School Budget"})

# add new calculated columns to new school dataset
school_df["Per Student Budget"] = school_df["Total School Budget"] / school_df["Total Students"]
school_df["Average Math Score"] = school_df["math_score"] / school_df["Total Students"]
school_df["Average Reading Score"] = school_df["reading_score"] / school_df["Total Students"]
school_df["% Passing Math"] = school_df["math_pass"] / school_df["Total Students"] * 100
school_df["% Passing Reading"] = school_df["reading_pass"] / school_df["Total Students"] * 100
school_df["Overall Passing Rate"] = (school_df["% Passing Math"] + school_df["% Passing Reading"]) / 2

# format columns
school_df["Total Students"] = school_df["Total Students"].astype(float).map("{:,.0f}".format)
school_df["Total School Budget"] = school_df["Total School Budget"].astype(float).map("${:,.0f}".format)
school_df["Per Student Budget"] = school_df["Per Student Budget"].astype(float).map("${:,.0f}".format)
school_df["Average Math Score"] = school_df["Average Math Score"].astype(float).map("{:.3f}".format)
school_df["Average Reading Score"] = school_df["Average Reading Score"].astype(float).map("{:.3f}".format)
school_df["% Passing Math"] = school_df["% Passing Math"].astype(float).map("{:.3f}%".format)
school_df["% Passing Reading"] = school_df["% Passing Reading"].astype(float).map("{:.3f}%".format)
school_df["Overall Passing Rate"] = school_df["Overall Passing Rate"].astype(float).map("{:.3f}%".format)

# set and format index
school_df.set_index("School Name", inplace=True)
school_df.index.name = None

# copy and prepare cleansed dataset to be used in next two steps (top / bottmom - sorting)
top_school_df = school_df.copy()
top_school_df.drop(inplace=True, columns=["math_score", "reading_score", "math_pass", "reading_pass"])


## Top Performing Schools (By Passing Rate)

In [4]:
top_school_df = top_school_df.sort_values(by=('Overall Passing Rate'), ascending=False)
top_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.062,83.976,94.133%,97.040%,95.587%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.418,83.849,93.272%,97.309%,95.291%
Pena High School,Charter,962,"$585,858",$609,83.840,84.045,94.595%,95.946%,95.270%
Griffin High School,Charter,"1,468","$917,500",$625,83.351,83.817,93.392%,97.139%,95.266%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.274,83.989,93.868%,96.540%,95.204%


## Bottom Performing Schools (By Passing Rate)

In [5]:
top_school_df = top_school_df.sort_values(by=('Overall Passing Rate'), ascending=True)
top_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.843,80.745,66.367%,80.220%,73.293%
Figueroa High School,District,"2,949","$1,884,411",$639,76.712,81.158,65.988%,80.739%,73.364%
Huang High School,District,"2,917","$1,910,635",$655,76.629,81.183,65.684%,81.316%,73.500%
Johnson High School,District,"4,761","$3,094,650",$650,77.072,80.966,66.058%,81.222%,73.640%
Ford High School,District,"2,739","$1,763,916",$644,77.103,80.746,68.310%,79.299%,73.804%


## Math Scores by Grade

In [6]:
# create dataset for each grade
scores_09 = student_data.loc[student_data['grade'] == "9th", ["school_name", "Student ID", "math_score", "reading_score"]]
scores_10 = student_data.loc[student_data['grade'] == "10th", ["school_name", "Student ID", "math_score", "reading_score"]]
scores_11 = student_data.loc[student_data['grade'] == "11th", ["school_name", "Student ID", "math_score", "reading_score"]]
scores_12 = student_data.loc[student_data['grade'] == "12th", ["school_name", "Student ID", "math_score", "reading_score"]]

# setup one data structure to be used for both Math & Reading later on
#    same steps are repeated for each grade (9-12)

# 9th
scores_09_grp = scores_09.groupby(["school_name"]).agg({"Student ID": "count", "math_score": "sum", "reading_score": "sum"})
scores_09_grp.reset_index(inplace=True)
scores_09_grp["9th Avg Reading"] = scores_09_grp["reading_score"] / scores_09_grp["Student ID"]
scores_09_grp["9th Avg Math"] = scores_09_grp["math_score"] / scores_09_grp["Student ID"]
scores_09_grp.sort_values(by=('school_name'), ascending=True)
scores_09_grp.drop(inplace=True, columns=["Student ID", "math_score", "reading_score"])

# 10th
scores_10_grp = scores_10.groupby(["school_name"]).agg({"Student ID": "count", "math_score": "sum", "reading_score": "sum"})
scores_10_grp.reset_index(inplace=True)
scores_10_grp["10th Avg Reading"] = scores_10_grp["reading_score"] / scores_10_grp["Student ID"]
scores_10_grp["10th Avg Math"] = scores_10_grp["math_score"] / scores_10_grp["Student ID"]
scores_10_grp.sort_values(by=('school_name'), ascending=True)
scores_10_grp.drop(inplace=True, columns=["Student ID", "math_score", "reading_score"])

# 11th
scores_11_grp = scores_11.groupby(["school_name"]).agg({"Student ID": "count", "math_score": "sum", "reading_score": "sum"})
scores_11_grp.reset_index(inplace=True)
scores_11_grp["11th Avg Reading"] = scores_11_grp["reading_score"] / scores_11_grp["Student ID"]
scores_11_grp["11th Avg Math"] = scores_11_grp["math_score"] / scores_11_grp["Student ID"]
scores_11_grp.sort_values(by=('school_name'), ascending=True)
scores_11_grp.drop(inplace=True, columns=["Student ID", "math_score", "reading_score"])

# 12th
scores_12_grp = scores_12.groupby(["school_name"]).agg({"Student ID": "count", "math_score": "sum", "reading_score": "sum"})
scores_12_grp.reset_index(inplace=True)
scores_12_grp["12th Avg Reading"] = scores_12_grp["reading_score"] / scores_12_grp["Student ID"]
scores_12_grp["12th Avg Math"] = scores_12_grp["math_score"] / scores_12_grp["Student ID"]
scores_12_grp.sort_values(by=('school_name'), ascending=True)
scores_12_grp.drop(inplace=True, columns=["Student ID", "math_score", "reading_score"])

# Create a new data set represeting all years merged together
scores_grp = pd.merge(scores_09_grp, scores_10_grp, how="inner", on=["school_name", "school_name"])
scores_grp = pd.merge(scores_grp, scores_11_grp, how="inner", on=["school_name", "school_name"])
scores_grp = pd.merge(scores_grp, scores_12_grp, how="inner", on=["school_name", "school_name"])

# set and format index column
scores_grp.set_index("school_name", inplace=True)
scores_grp.index.name = None

# create new dataset based common one for reading and math
math = scores_grp.copy()
read = scores_grp.copy()

In [7]:
# remove reading columns
math.drop(inplace=True, columns=["9th Avg Reading", "10th Avg Reading", "11th Avg Reading", "12th Avg Reading"])

# rename columns to generic values
math.rename(inplace=True, columns={
    "9th Avg Math": "9th", 
    "10th Avg Math": "10th",
    "11th Avg Math": "11th",
    "12th Avg Math": "12th"})

# display results
math

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [8]:
# remove reading columns
read.drop(inplace=True, columns=["9th Avg Math", "10th Avg Math", "11th Avg Math", "12th Avg Math"])

# rename columns to generic values
read.rename(inplace=True, columns={
    "9th Avg Reading": "9th", 
    "10th Avg Reading": "10th",
    "11th Avg Reading": "11th",
    "12th Avg Reading": "12th"})

# display results
read

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [9]:
# make a copy of complete merged data from earlier
bin_df = school_data_complete.copy()

# drop unneeded columns
bin_df.drop(inplace=True, columns=["Student ID", "student_name", "gender", "grade","school_name", "School ID"])

In [10]:
# use a copy to preserve bin_df for other sections
bin1_df = bin_df.copy()

# Create bins and labels
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# add calculated column for student budget
bin1_df["per_student_budget"] = bin1_df["budget"] / bin1_df["size"]

# drop unneeded columns
bin1_df.drop(inplace=True, columns=["type", "size", "budget"])

# add new column based on bin ranges and a column to allow for easy sum/count within each group
bin1_df["Spending Range"] = pd.cut(bin1_df["per_student_budget"], spending_bins, labels=group_names)
bin1_df["Total Students"] = 1

# create new dataset based on grouping
bin1_grp = bin1_df.groupby("Spending Range").agg({"math_score": "sum",
                                               "reading_score": "sum",
                                               "math_pass": "sum", 
                                               "reading_pass": "sum", 
                                               "Total Students": "sum"})

# add new calculated columns to the data set
bin1_grp["Average Math Score"] = bin1_grp["math_score"] / bin1_grp["Total Students"]
bin1_grp["Average Reading Score"] = bin1_grp["reading_score"] / bin1_grp["Total Students"]
bin1_grp["% Passing Math"] = bin1_grp["math_pass"] / bin1_grp["Total Students"] * 100
bin1_grp["% Passing Reading"] = bin1_grp["reading_pass"] / bin1_grp["Total Students"] * 100
bin1_grp["Overall Passing Rate"] = (bin1_grp["% Passing Math"] + bin1_grp["% Passing Reading"]) / 2

# remove unneeded columns (those used to derive calcualted fields)
bin1_grp.drop(inplace=True, columns=["math_score", "reading_score", "math_pass", "reading_pass", "Total Students"])

# dispaly results
bin1_grp.head()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Range,,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

In [11]:
# use a copy to preserve bin_df for other sections
bin2_df = bin_df.copy()

# Create bins and labels
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# drop unneeded columns
bin2_df.drop(inplace=True, columns=["type", "budget"])

# add new column based on bin ranges
bin2_df["School Size"] = pd.cut(bin2_df["size"], size_bins, labels=group_names)
bin2_df["Total Students"] = 1

# create new dataset based on grouping
bin2_grp = bin2_df.groupby("School Size").agg({"math_score": "sum",
                                               "reading_score": "sum",
                                               "math_pass": "sum", 
                                               "reading_pass": "sum", 
                                               "Total Students": "sum"})

# add new calculated columns to new school dataset
bin2_grp["Average Math Score"] = bin2_grp["math_score"] / bin2_grp["Total Students"]
bin2_grp["Average Reading Score"] = bin2_grp["reading_score"] / bin2_grp["Total Students"]
bin2_grp["% Passing Math"] = bin2_grp["math_pass"] / bin2_grp["Total Students"] * 100
bin2_grp["% Passing Reading"] = bin2_grp["reading_pass"] / bin2_grp["Total Students"] * 100
bin2_grp["Overall Passing Rate"] = (bin2_grp["% Passing Math"] + bin2_grp["% Passing Reading"]) / 2

# remove unneeded columns (those used to derive the calculated fields)
bin2_grp.drop(inplace=True, columns=["math_score", "reading_score", "math_pass", "reading_pass", "Total Students"])

# display results
bin2_grp.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

In [12]:
# use a copy to preserve bin_df for other sections
bin3_df = bin_df.copy()

# drop unneeded columns
bin3_df.drop(inplace=True, columns=["size", "budget"])

# add new column based on bin ranges
bin3_df["Total Students"] = 1

# create new dataset based on grouping
bin3_grp = bin3_df.groupby("type").agg({"math_score": "sum",
                                               "reading_score": "sum",
                                               "math_pass": "sum", 
                                               "reading_pass": "sum", 
                                               "Total Students": "sum"})

#add new calculated columns to new school dataset
bin3_grp["Average Math Score"] = bin3_grp["math_score"] / bin3_grp["Total Students"]
bin3_grp["Average Reading Score"] = bin3_grp["reading_score"] / bin3_grp["Total Students"]
bin3_grp["% Passing Math"] = bin3_grp["math_pass"] / bin3_grp["Total Students"] * 100
bin3_grp["% Passing Reading"] = bin3_grp["reading_pass"] / bin3_grp["Total Students"] * 100
bin3_grp["Overall Passing Rate"] = (bin3_grp["% Passing Math"] + bin3_grp["% Passing Reading"]) / 2

# remove unneeded columns (those used to derive the calculated fields)
bin3_grp.drop(inplace=True, columns=["math_score", "reading_score", "math_pass", "reading_pass", "Total Students"])

# rename index column
bin3_grp.index.name = "School Type"

# show results
bin3_grp.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
